# Pomysł
Zrobić wykres https://datavizproject.com/data-type/proportional-area-chart/ korzystając z danych https://en.wikipedia.org/wiki/List_of_highest-grossing_media_franchises ew z tego https://en.wikipedia.org/wiki/List_of_best-selling_video_game_franchises 